In [1]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os
import subprocess
import shutil


In [2]:
def apply_request_patch():
    """Apply workaround for the 403 Forbidden error"""
    
    import pytubefix.request
    # Store the original request function
    original_request = pytubefix.request._execute_request
    
    # Create a patched version with browser-like headers
    def patched_request(url, method=None, headers=None, data=None, timeout=30):
        if headers is None:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
                'Accept-Language': 'en-US,en;q=0.9',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Referer': 'https://www.youtube.com/'
            }
        return original_request(url, method=method, headers=headers, data=data, timeout=timeout)
    
    # Replace the original with our patched version
    pytubefix.request._execute_request = patched_request
    print("Applied YouTube download workaround")

# Apply the patch
apply_request_patch()

Applied YouTube download workaround


In [ ]:
def format_size(bytes):
    """Format file size in human-readable format"""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if bytes < 1024:
            return f"{bytes:.2f} {unit}"
        bytes /= 1024
    return f"{bytes:.2f} TB"

def create_safe_filename(title):
    """Create a safe filename from video title"""
    # Remove invalid characters for filenames
    invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
    for char in invalid_chars:
        title = title.replace(char, '')
    # Limit length to avoid path too long errors
    return title[:100]



def merge_video_audio(video_path, audio_path, output_path):
    """Merge video and audio using FFmpeg"""
    try:
        # Use shell=True to let Windows find ffmpeg in the PATH
        cmd = f'ffmpeg -i "{video_path}" -i "{audio_path}" -c:v copy -c:a aac -strict experimental "{output_path}"'
        subprocess.run(cmd, shell=True, check=True)
        return True
    except subprocess.SubprocessError as e:
        print(f"Error merging files: {str(e)}")
        return False

YouTube Video URL

In [4]:
url = "https://www.youtube.com/watch?v=HicQ828PSvM&list=WL&index=8"
download_dir = r"C:\Projects\project01_ytdownload\downloads" 
os.makedirs(download_dir, exist_ok=True)


In [5]:
# Create YouTube object with progress callback
yt = YouTube(url, on_progress_callback=on_progress)
safe_title = create_safe_filename(yt.title)

# Print video information
print(f"Title: {yt.title}")
print(f"Channel: {yt.author}")
print(f"Length: {yt.length} seconds")
print(f"Views: {yt.views:,}")
print(f"Download directory: {download_dir}")

# Get progressive streams (includes both video and audio)
progressive_streams = yt.streams.filter(progressive=True).order_by('resolution')

# Print available progressive qualities
print("\nAvailable progressive video qualities (video+audio combined):")
for i, stream in enumerate(progressive_streams, 1):
    file_size = format_size(stream.filesize)
    print(f"{i}. Resolution: {stream.resolution}, "
          f"Format: {stream.mime_type}, "
          f"Size: {file_size}, "
          f"FPS: {stream.fps}")

# Get adaptive streams (higher quality, video only)
print("\nHigher quality video-only streams (requires separate audio):")
video_streams = yt.streams.filter(adaptive=True, only_video=True,file_extension='mp4').order_by('resolution')
for i, stream in enumerate(video_streams, 1):
    file_size = format_size(stream.filesize)
    print(f"{i}. Resolution: {stream.resolution}, "
          f"Format: {stream.mime_type}, "
          f"Size: {file_size}, "
          f"FPS: {stream.fps}")

# Print audio-only options
print("\nAudio-only options:")
audio_streams = yt.streams.filter(only_audio=True,file_extension='mp4').order_by('abr')
for i, stream in enumerate(audio_streams, 1):
    file_size = format_size(stream.filesize)
    print(f"{i}. Bitrate: {stream.abr}, "
          f"Format: {stream.mime_type}, "
          f"Size: {file_size}")

# Get the best video and audio streams separately
best_video = yt.streams.filter(adaptive=True, only_video=True,file_extension='mp4').order_by('resolution').last()
best_audio = yt.streams.filter(only_audio=True,file_extension='mp4').order_by('abr').last()

print(f"\nBest video quality available: {best_video.resolution} ({format_size(best_video.filesize)})")
print(f"Best audio quality available: {best_audio.abr} ({format_size(best_audio.filesize)})")




Title: دوره القای نقدینگی LIT قسمت نهم: دو نوع اوردربلاک مهم.
Channel: AGORA
Length: 888 seconds
Views: 1,500
Download directory: C:\Projects\project01_ytdownload\downloads

Available progressive video qualities (video+audio combined):
1. Resolution: 360p, Format: video/mp4, Size: 21.54 MB, FPS: 25

Higher quality video-only streams (requires separate audio):
1. Resolution: 144p, Format: video/mp4, Size: 3.16 MB, FPS: 25
2. Resolution: 240p, Format: video/mp4, Size: 6.17 MB, FPS: 25
3. Resolution: 360p, Format: video/mp4, Size: 11.61 MB, FPS: 25
4. Resolution: 480p, Format: video/mp4, Size: 19.65 MB, FPS: 25
5. Resolution: 720p, Format: video/mp4, Size: 34.97 MB, FPS: 25
6. Resolution: 1080p, Format: video/mp4, Size: 57.99 MB, FPS: 25

Audio-only options:
1. Bitrate: 48kbps, Format: audio/mp4, Size: 5.16 MB
2. Bitrate: 128kbps, Format: audio/mp4, Size: 13.70 MB

Best video quality available: 1080p (57.99 MB)
Best audio quality available: 128kbps (13.70 MB)


In [6]:
# Download highest progressive stream (video+audio but lower quality)
print("\nDownloading highest progressive quality...")

fileName = safe_title
safe_title = "vid"


# highest_progressive = yt.streams.get_highest_resolution()
# progressive_file = highest_progressive.download(
#     output_path=download_dir,
#     filename=f"{safe_title}_progressive.{highest_progressive.subtype}"
# )
# print(f"Downloaded: {os.path.basename(progressive_file)}")

# Download highest quality video and audio separately
print("\nDownloading highest quality video...")
video_file = best_video.download(
    output_path=download_dir,
    filename=f"{safe_title}_video.{best_video.subtype}"
)
print(f"Downloaded video: {os.path.basename(video_file)}")

print("Downloading highest quality audio...")
audio_file = best_audio.download(
    output_path=download_dir,
    filename=f"{safe_title}_audio.{best_audio.subtype}"
)
print(f"Downloaded audio: {os.path.basename(audio_file)}")





Downloaded video: vid_video.mp4█████████████████| 100.0%
Downloaded audio: vid_audio.mp4█████████████████| 100.0%


In [7]:
path_ffmpeg = r"C:\ffmpeg\ffmpeg-master-latest-win64-gpl-shared\bin\ffmpeg.exe"

In [8]:
def merge_video_audio(video_path, audio_path, output_path):
    """Merge video and audio using FFmpeg with full path"""
    try:
        # Use the full path to FFmpeg
        cmd = f'"{path_ffmpeg}" -i "{video_path}" -i "{audio_path}" -c:v copy -c:a copy "{output_path}"'
        
        print(f"Running command: {cmd}")
        result = subprocess.run(
            cmd, 
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        if result.returncode != 0:
            print(f"Error output: {result.stderr[:200]}...")
            return False
            
        return True
    except Exception as e:
        print(f"Error during merge: {str(e)}")
        return False

In [ ]:
# Check if FFmpeg is available and try to merge
merged_file = os.path.join(download_dir, f"{safe_title}_merged.mp4")
ffmpeg_available = is_ffmpeg_available()

print("\n--- MERGING INSTRUCTIONS ---")
ffmpeg_available = True

    if merge_video_audio(video_file, audio_file, merged_file):
        print(f"Successfully merged files to: {os.path.basename(merged_file)}")
        print("\nYou now have two files:")
        
        print(f"1. {os.path.basename(merged_file)} - High quality merged file")
        print(f"2. Individual video and audio files (can be deleted if merge was successful)")
    else:
        print("Automatic merge failed. See manual instructions below.")
        print_manual_instructions = True
else:
    print("FFmpeg not found. To merge the high-quality video and audio files, you need to install FFmpeg.")
    
print("\n--- FILES CREATED ---")
print(f"1. Progressive video: {progressive_file}")
print(f"2. High-quality video: {video_file}")
print(f"3. Audio: {audio_file}")
if ffmpeg_available and os.path.exists(merged_file):
    print(f"4. Merged high-quality file: {merged_file}")

print("\nAll files saved to:", download_dir)


--- MERGING INSTRUCTIONS ---
FFmpeg detected! Attempting automatic merge...
Running command: "C:\ffmpeg\ffmpeg-master-latest-win64-gpl-shared\bin\ffmpeg.exe" -i "C:\Projects\project01_ytdownload\downloads\vid_video.mp4" -i "C:\Projects\project01_ytdownload\downloads\vid_audio.mp4" -c:v copy -c:a copy "C:\Projects\project01_ytdownload\downloads\vid_merged.mp4"


In [ ]:
def rename_and_cleanup(merged_file, video_file, audio_file, new_name):
    """
    Rename the merged file with a new name (keeping extension) and delete source files
    
    Args:
        merged_file: Path to the merged file
        video_file: Path to the video file to delete
        audio_file: Path to the audio file to delete
        new_name: New base filename (without extension)
    
    Returns:
        Path to the renamed file
    """
    try:
        # Make sure merged file exists
        if not os.path.exists(merged_file):
            print(f"Error: Merged file {merged_file} not found")
            return None
            
        # Get directory and extension from the merged file
        directory = os.path.dirname(merged_file)
        _, extension = os.path.splitext(merged_file)
        
        # Create new filename with original extension
        new_file_path = os.path.join(directory, f"{new_name}{extension}")
        
        # Rename the file
        os.rename(merged_file, new_file_path)
        print(f"Renamed merged file to: {os.path.basename(new_file_path)}")
        
        # Delete the source files
        if os.path.exists(video_file):
            os.remove(video_file)
            print(f"Deleted: {os.path.basename(video_file)}")
            
        if os.path.exists(audio_file):
            os.remove(audio_file)
            print(f"Deleted: {os.path.basename(audio_file)}")
            
        return new_file_path
    except Exception as e:
        print(f"Error during rename/cleanup: {str(e)}")
        return None

# Example usage:
new_filename = fileName  # This is your variable containing the new name

if os.path.exists(merged_file):
    final_path = rename_and_cleanup(merged_file, video_file, audio_file, new_filename)
    if final_path:
        print(f"Final video saved as: {final_path}")
else:
    print("Merged file not found, cannot rename/cleanup")